In [64]:
from langchain.embeddings import OllamaEmbeddings, HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Pinecone, Chroma, FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.llms import CTransformers, Ollama
from langchain_pinecone import PineconeVectorStore
import pinecone

In [65]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [66]:
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")

In [67]:
#DATA INGESTION
def load_pdf(data):
    loader = DirectoryLoader(data, glob='*.pdf', loader_cls=PyPDFLoader)
    docs = loader.load()
    return docs

In [68]:
extracted_data = load_pdf(data="D:\My_Coding_Files\medical-chatbot-using-llama2\data")

<>:1: SyntaxWarning: invalid escape sequence '\M'
<>:1: SyntaxWarning: invalid escape sequence '\M'
C:\Users\Arjo\AppData\Local\Temp\ipykernel_8576\2692790921.py:1: SyntaxWarning: invalid escape sequence '\M'
  extracted_data = load_pdf(data="D:\My_Coding_Files\medical-chatbot-using-llama2\data")


In [69]:
len(extracted_data)

637

In [70]:
#DATA TRANSFORMATION
#CREATING TEXT CHUNKS
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [71]:
docs = text_split(extracted_data=extracted_data)

In [72]:
len(docs)

7020

In [73]:
#Use embedding model
def hugging_face_embed_docs():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

def ollama_embed_docs():
    embeddings = OllamaEmbeddings(model="llama2")
    return embeddings

In [93]:
embeddings = ollama_embed_docs()
hugging_embeddings = hugging_face_embed_docs()

d:\My_Coding_Files\medical-chatbot-using-llama2\medchat\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Arjo\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
d:\My_Coding_Files\medical-chatbot-using-llama2\medchat\Lib\site-packag

In [75]:
embeddings

OllamaEmbeddings(base_url='http://localhost:11434', model='llama2', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=False, headers=None, model_kwargs=None)

In [94]:
hugging_embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [86]:
index_name = "medical-chatbot-ollama-llama2"

In [78]:
# from pinecone import Pinecone

# pc = Pinecone(api_key=PINECONE_API_KEY)

In [88]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

In [89]:
index = pc.Index(index_name)

In [95]:
vector_store = FAISS.from_documents(docs, hugging_embeddings)

In [96]:
vector_store

In [99]:
sample_ans = vector_store.similarity_search("What is an allergy", k=3)
sample_ans

[Document(page_content='When thisoccurs, an allergy develops against the offending sub-stance (an allergen.)', metadata={'source': 'D:\\My_Coding_Files\\medical-chatbot-using-llama2\\data\\Medical_book.pdf', 'page': 128, 'text': 'When thisoccurs, an allergy develops against the offending sub-stance (an allergen.)'}),
 Document(page_content='Purpose\nAllergy is a reaction of the immune system. Nor-', metadata={'source': 'D:\\My_Coding_Files\\medical-chatbot-using-llama2\\data\\Medical_book.pdf', 'page': 135, 'text': 'Purpose\nAllergy is a reaction of the immune system. Nor-'}),
 Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\

In [114]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don;t try to make up an answer.

Context: {context}
Question: {question}
Only return the helpful answer below and nothing else
Helpful answer : 
"""

In [115]:
PROMPT = PromptTemplate(template = prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt" : PROMPT}

In [112]:
llm = CTransformers(model = r"C:\Users\Arjo\.ollama\models\blobs\sha256-8934d96d3f08982e95922b2b7a2c626a1fe873d7c3b06e8e56d7bc0a1fef9246", model_type='llama2', config={'max_new_tokens' : 512, 'temperature' : 0.8})

In [117]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = 'stuff',
    chain_type_kwargs = chain_type_kwargs,
    return_source_documents = True,
    retriever = vector_store.as_retriever(search_kwargs = {'k' : 2})
)

In [118]:
while True:
    user_input = input(f"Input Prompt : ")
    result = qa({'query' : user_input})
    print("Response : ", result['result'])

d:\My_Coding_Files\medical-chatbot-using-llama2\medchat\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Response :  Fallopian tube is not mentioned in the passage provided. Therefore, I do not know the answer to your question.
Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
Response :  You have previously inputted the term "KEY TERMS".


KeyboardInterrupt: Interrupted by user